In [ ]:
import timm
import cv2
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

In [ ]:
class Classifier(nn.Module):
    def __init__(self, name, *, pretrained=False, in_chans=3):
        super().__init__()
        model = timm.create_model(name, pretrained=pretrained, in_chans=in_chans)
        clsf = model.default_cfg['classifier']
        n_features = model._modules[clsf].in_features
        model._modules[clsf] = nn.Identity()
        
        self.fc = nn.Linear(n_features, 1)
        self.model = model
    
    def forward(self, x):
        x = self.model(x)
        logits = self.fc(x)
        return logits

In [ ]:
model = Classifier('tf-efficientnetv2_s', pretrained=False)
model.to(device)
model.load_state_dict(torch.load('./efficientnetv2_s.pth', map_location=device)['model'])

def inference(img_path, model):
    model.eval()
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256), interpolation =cv2.INTER_LINEAR) / 255
    img = torch.tensor(img, dtype=torch.float32).to(device).reshape([-1, 3, 256, 256])
    pred = model(img).squeeze().sigmoid().detach().cpu().numpy()
    print(pred)

In [ ]:
path = './samples/avatar-1122482d4efbd3f44e4c99e93b36d42d.jpg'
inference(path, model)
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)